<a href="https://colab.research.google.com/github/KoukiNishihara/AccessLogAnalyzer/blob/master/AccessLogAnalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# git clone していない場合の実行





In [76]:
!git clone https://github.com/KoukiNishihara/AccessLogAnalyzer.git

fatal: destination path 'AccessLogAnalyzer' already exists and is not an empty directory.


In [0]:
%cd AccessLogAnalyzer/



# git clone 済みの場合の実行




In [78]:
!pip3 install apache-log-parser

In [0]:
import datetime
import apache_log_parser

In [0]:
#これを実行するとプログラムが始まる
def Main():
    #諸々の初期化
    filename=['access_log'] #ファイル名
    host_dict={}        #ホスト名とアクセス数を格納するもの
    time_list=[]        #単位時間あたりのアクセス数
    line_parser = apache_log_parser.make_parser("%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"")
    file_num=0          #解析したファイル数
    time_range=[]       #解析期間

    host_dict,time_list,time_range,file_num=Analyze(filename,host_dict,time_list,time_range,file_num)   #ログの解析
    Output(host_dict,time_list,time_range,file_num)     #出力

In [0]:
#入力を促し，ログの解析をする
def Analyze(filename,host_dict,time_list,time_range,file_num):
    time_range=input('ログを収集する期間を指定してください.指定がない場合はそのままEnterをおしてください\n例）2000/11/06 2017/11/07\n').split()

    # 0時から23時までを0で初期化
    for i in range(0,24):
        time_list.append(0)

    file_num=FileRead(filename,host_dict,time_list,time_range)
    print()

    filename=input('"access_log"以外に解析したいファイルがあればファイル名を入力してください．指定がない場合はそのままEnterをおしてください.\n例）ファイル名\n例）ファイル名1　ファイル名2　・・・　ファイル名n\n').split()

    file_num+=FileRead(filename,host_dict,time_list,time_range)
    return  host_dict,time_list,time_range,file_num

In [0]:
#ファイル読み込み
def FileRead(filenames=['access_log'],host_dict={},time_list=[],time_range=[]):
    file_num=0
    if(len(time_range)==0 or len(time_range)==2):   #解析期間の指定がない場合，もしくは初めの期間と終わりの期間が指定された場合

        if(len(time_range)==2):         #初めの期間と終わりの期間が指定された場合
            start=time_range[0]
            end=time_range[1]

        for name in filenames:  
            file_num+=1             #解析するファイル数のカウント

            with open(name,'r') as fh:      #ファイル読み込み
                line =fh.readline()         #一行づつ読み込む

                while line:
                    log_line_data = line_parser(line)
                    date=log_line_data["time_received_datetimeobj"].strftime("%Y/%m/%d")

                    if((len(time_range)==0) or (start<=date and end>=date)):    #解析期間内である場合
                        CountHost(log_line_data,host_dict)      #host名とアクセス数をカウントする
                        CountPerHour(log_line_data,time_list)   #単位時間あたりのアクセス数をカウント
                    line =fh.readline()
    return file_num

In [0]:
#　hostのアクセス数をカウント
def CountHost(log_line_data,host_dict):

    host_name=log_line_data["remote_host"]

    if host_name not in host_dict.keys():
        i=1
        update_data = {host_name:i}
        host_dict.update(update_data)

    else:
        host_dict[host_name]+=1

    return host_dict

In [0]:
# 単位時間あたりのカウント
def CountPerHour(log_line_data,time_list):

    time_log=log_line_data['time_received_datetimeobj'].hour
    time_list[time_log]+=1

    return host_dict

In [0]:
#解析したものを出力
def Output(host_dict,time_list,time_range,file_num):
    print("解析したファイルは",file_num,"個です")

    if(len(host_dict)==0):
        print("アクセスはありませんでした")
    else :
        if(len(time_range)==2):
            print("解析期間は",time_range[0],"から",time_range[1],"です.")
        elif(len(time_range)==0):
                print("解析期間は全期間です.")

    print()
    for k,v in sorted(host_dict.items(),key=lambda x:x[1],reverse=True):
        print("host:",k,"count:",v)
    print("------------------------------")

    i=0
    for n in time_list:
        print("hour:",i,"cout:",n)
        i+=1    



*   解析期間を指定せず他のファイルを指定してしなかった場合



In [99]:
Main()

ログを収集する期間を指定してください.指定がない場合はそのままEnterをおしてください
例）2000/11/06 2017/11/07


"access_log"以外に解析したいファイルがあればファイル名を入力してください．指定がない場合はそのままEnterをおしてください.
例）ファイル名
例）ファイル名1　ファイル名2　・・・　ファイル名n

解析したファイルは 1 個です
解析期間は全期間です.

host: 10.2.3.4 count: 3
host: 10.2.3.5 count: 1
host: 10.2.3.6 count: 1
------------------------------
hour: 0 cout: 2
hour: 1 cout: 0
hour: 2 cout: 0
hour: 3 cout: 1
hour: 4 cout: 0
hour: 5 cout: 0
hour: 6 cout: 0
hour: 7 cout: 0
hour: 8 cout: 0
hour: 9 cout: 0
hour: 10 cout: 1
hour: 11 cout: 0
hour: 12 cout: 0
hour: 13 cout: 0
hour: 14 cout: 0
hour: 15 cout: 0
hour: 16 cout: 0
hour: 17 cout: 0
hour: 18 cout: 0
hour: 19 cout: 0
hour: 20 cout: 1
hour: 21 cout: 0
hour: 22 cout: 0
hour: 23 cout: 0




*  解析期間を指定せず，他のファイル指定した場合



In [102]:
Main()

ログを収集する期間を指定してください.指定がない場合はそのままEnterをおしてください
例）2000/11/06 2017/11/07


"access_log"以外に解析したいファイルがあればファイル名を入力してください．指定がない場合はそのままEnterをおしてください.
例）ファイル名
例）ファイル名1　ファイル名2　・・・　ファイル名n
other_log
解析したファイルは 2 個です
解析期間は全期間です.

host: 10.2.3.4 count: 3
host: xxx.xxx.xxx.xxx count: 2
host: 10.2.3.5 count: 1
host: 10.2.3.6 count: 1
------------------------------
hour: 0 cout: 2
hour: 1 cout: 0
hour: 2 cout: 0
hour: 3 cout: 1
hour: 4 cout: 0
hour: 5 cout: 0
hour: 6 cout: 0
hour: 7 cout: 0
hour: 8 cout: 0
hour: 9 cout: 0
hour: 10 cout: 1
hour: 11 cout: 0
hour: 12 cout: 0
hour: 13 cout: 0
hour: 14 cout: 0
hour: 15 cout: 0
hour: 16 cout: 0
hour: 17 cout: 0
hour: 18 cout: 0
hour: 19 cout: 0
hour: 20 cout: 1
hour: 21 cout: 0
hour: 22 cout: 0
hour: 23 cout: 2


* 期間指定を指定して，他のファイルを指定した場合





In [103]:
Main()

ログを収集する期間を指定してください.指定がない場合はそのままEnterをおしてください
例）2000/11/06 2017/11/07
2000/11/06 2017/11/07

"access_log"以外に解析したいファイルがあればファイル名を入力してください．指定がない場合はそのままEnterをおしてください.
例）ファイル名
例）ファイル名1　ファイル名2　・・・　ファイル名n
other_log
解析したファイルは 2 個です
解析期間は 2000/11/06 から 2017/11/07 です.

host: 10.2.3.4 count: 3
host: 10.2.3.5 count: 1
host: 10.2.3.6 count: 1
------------------------------
hour: 0 cout: 2
hour: 1 cout: 0
hour: 2 cout: 0
hour: 3 cout: 1
hour: 4 cout: 0
hour: 5 cout: 0
hour: 6 cout: 0
hour: 7 cout: 0
hour: 8 cout: 0
hour: 9 cout: 0
hour: 10 cout: 1
hour: 11 cout: 0
hour: 12 cout: 0
hour: 13 cout: 0
hour: 14 cout: 0
hour: 15 cout: 0
hour: 16 cout: 0
hour: 17 cout: 0
hour: 18 cout: 0
hour: 19 cout: 0
hour: 20 cout: 1
hour: 21 cout: 0
hour: 22 cout: 0
hour: 23 cout: 0
